In [1]:
import pandas as pd
import numpy as np
import math
import time
from datetime import datetime
from datetime import date, timedelta
import calendar
import re
from dateutil.relativedelta import relativedelta
from dateutil import parser

In [2]:
data = pd.read_csv("./NeoGrowth_CRIF format_17-04-2023_Output/Account.csv",sep='|',index_col=False)
data = data.rename(columns=lambda x: x.strip())
data = data.rename(columns = {'REFERENCE_NO':'LOS-APP-ID'})
data['CREDIT-GRANTOR']  = np.where(data['SELF-INDICATOR'] != True,'XXXX', 'Neogrowth') 

data.to_csv('Input_data.csv',index = False)
print("----------Input file generated---------")

----------Input file generated---------


In [3]:
data.shape[0]

1909

In [4]:
data['LOS-APP-ID'] = data['LOS-APP-ID'].astype('str')

In [5]:
test = data[['LOS-APP-ID']]
test = test.drop_duplicates()
x = test.shape[0]
x

54

In [6]:
def udf_add_months(date_in, n):
    #print(date_in, n,str(type(date_in)))
    if 'datetime.datetime' not in str(type(date_in)) or 'timestamps.Timestamp' not in str(type(date_in)):
        date_in = parser.parse(date_in)
    if math.isnan(n) == 1:
        date_out = np.nan
    else:
        n = int(n/3)
        date_in = pd.to_datetime(date_in, format="%d/%m/%y",infer_datetime_format=True, errors='ignore')
        #adding months to date_in
        date_out = date_in + relativedelta(months=n)
        date_out = date_out.strftime('%d/%m/%Y')
    return date_out

def udf_dpd(str_in):
    n = 3
    split_string = [str(str_in)[i:i+n] for i in range(0,len(str(str_in)),n)]
    if split_string == '' or len(split_string) == 0:
        return 0
    else:
        for i in range(len(split_string)):
            if split_string[i] == 'DDD':
                split_string[i] = '-100'
            elif split_string[i] == 'XXX':
                split_string[i] = '-200'
            else:
                split_string[i] = split_string[i]
            
        #split_string = [ i.rstrip('0') for i in split_string ]
        
        out = max(split_string)
    
        return out
    
def udf_dpd_months(str_in, n):
    if n == 0:
        return 0
    else: 
        out = str(str_in)[0:n]
        return out
    
def udf_dpd_30(str_in):
    n = 3
    out = 0
    split_string = [str(str_in)[i:i+n] for i in range(0,len(str(str_in)),n)]
    if split_string == '':
        return 0
    else:
        for i in range(len(split_string)):
            if split_string[i] == 'DDD':
                split_string[i] = '-100'
            elif split_string[i] == 'XXX':
                split_string[i] = '-200'
            else:
                split_string[i] = split_string[i]
        try:
            split_string = [ i.replace('nan','0') for i in split_string ]
            for i in range(len(split_string)):
                if split_string[i] != '':
                    if int(split_string[i]) > 30:
                        split_string[i] = '1'
                    else:
                        split_string[i] = '0'
                else:
                    split_string[i] = '0'
                out = out + int(split_string[i])
        except:
            print(str_in)
    
        return out
        
def udf_dpd_bad(str_in):
    n = 3
    out = 0
    split_string = [str(str_in)[i:i+n] for i in range(0,len(str(str_in)),n)]
    if split_string == '':
        return 0
    else:
        for i in range(len(split_string)):
            if split_string[i] == 'DDD':
                split_string[i] = '-100'
            elif split_string[i] == 'XXX':
                split_string[i] = '-200'
            else:
                split_string[i] = split_string[i]
        try:
            split_string = [ i.replace('nan','0') for i in split_string ]
            for i in range(len(split_string)):
                if split_string[i] != '':
                    if int(split_string[i]) > 0:
                        split_string[i] = '1'
                    else:
                        split_string[i] = '0'
                else:
                    split_string[i] = '0'
                out = out + int(split_string[i])
        except:
            print(str_in)
    
        return out


In [7]:
strng = 'S06S06S05S06S06S06S06S06S06S06S06S05S05S05S05S05S05S04S04S04S04S05S05S05S05S05S05S05S05S05S04S04S04S04S04S04'
index = strng.index('S06')+1
month = int(index/3)
month

0

In [8]:
#WRITE-OFF-AMT - amount can't be collected from customer/ customer unable to pay this amount.
data['write_off'] = np.where(data['WRITE-OFF-AMT'] > 500, 1, 0)

data['b'] = data['DAS - HIST'].str.find('S06') + 1

#Adding data['b'] value with the date-reported month
data['DATE-REPORTED1'] = data.apply(lambda x: udf_add_months(x['DATE-REPORTED'], x['b']), 
                        axis=1)
              
data['write_off_date1_own'] = np.where((data['b'] == 0)&(data['CREDIT-GRANTOR'] != 'XXXX'), data['CLOSE-DT'], data['DATE-REPORTED'])

data['write_off_date1_other'] = np.where((data['b'] == 0)&(data['CREDIT-GRANTOR'] != 'XXXX'), data['CLOSE-DT'], data['DATE-REPORTED'])


# data['a'] = data['DAS - HIST'].str.find('S16') + 1
# data['DATE-REPORTED2'] = data.apply(lambda x: udf_add_months(x['DATE-REPORTED'], x['a']), 
#                         axis=1)

# data['suit_filed_date_own'] = np.where((data['a'] != 0)&(data['CREDIT-GRANTOR'] != 'XXXX'), data['DATE-REPORTED2'], np.nan)
# data['suit_filed_date_other'] = np.where((data['a'] != 0)&(data['CREDIT-GRANTOR'] == 'XXXX'), data['DATE-REPORTED2'], np.nan)


In [9]:
#Fetching unique loan_ids.
summary = data[['LOS-APP-ID']]
summary = summary.drop_duplicates()

#formating date-reported column
data['DATE-REPORTED'] = pd.to_datetime(data['DATE-REPORTED'], format="%d/%m/%y",infer_datetime_format=True, errors='ignore')

#get starting year from date-reported column by groupby.
df1 = data.groupby('LOS-APP-ID', as_index=False)['DATE-REPORTED'].min()
df1['min_year'] = pd.DatetimeIndex(df1['DATE-REPORTED']).year
#merge loanid df and grouped date-reported data
summary = summary.merge(df1, on='LOS-APP-ID', how='left')

#get todays date and find the year gap with date-reported.
summary['today_date'] = pd.to_datetime('today').strftime("%Y-%m-%d")
summary['CREDIT_HISTORY'] = pd.DatetimeIndex(summary['today_date']).year - pd.DatetimeIndex(summary['DATE-REPORTED']).year


/Users/aleena.varghese/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '28-02-2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/Users/aleena.varghese/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '31-08-2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/Users/aleena.varghese/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '31-03-2023' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/Users/aleena.varghese/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.p

In [10]:
#written off
data['count']=data['DAS - HIST'].str.contains('S06')

# Sometimes less amount will be there so need to check amount instaed of 's06'
data['WRITE_OFF_OWN'] = np.where((data['CREDIT-GRANTOR'] != 'XXXX') & (data['WRITE-OFF-AMT'] > 500), 1, 0)
data['WRITE_OFF_OTHERS'] = np.where((data['CREDIT-GRANTOR'] == 'XXXX') & (data['WRITE-OFF-AMT'] > 500), 1, 0)

#-----------
#find the first matching index location of 'S06' from 'DAS - HIST'.if there is no matching then the value will be (-1)+1 = 0
data['w'] = data['DAS - HIST'].str.find('S06') + 1

#Adding data['b'] value with the date-reported month
data['DATE-REPORTED_w'] = data.apply(lambda x: udf_add_months(str(x['DATE-REPORTED']), x['w']), 
                        axis=1)
# The below logic is valied need to check with someone 
# data['write_off_date1_own'] = np.where((data['w'] == 0)&(data['CREDIT-GRANTOR'] != 'XXXX'), data['CLOSE-DT'], data['DATE-REPORTED'])

# data['write_off_date1_other'] = np.where((data['w'] == 0)&(data['CREDIT-GRANTOR'] != 'XXXX'), data['CLOSE-DT'], data['DATE-REPORTED'])


data['write_off_date1_own'] = np.where((data['w'] != 0) & (data['CREDIT-GRANTOR'] != 'XXXX'), data['DATE-REPORTED_w'], np.nan)

data['write_off_date1_other'] = np.where((data['w'] != 0) & (data['CREDIT-GRANTOR'] == 'XXXX'), data['DATE-REPORTED_w'], np.nan)

df5 = data.sort_values('write_off_date1_own').groupby('LOS-APP-ID').head(1)[['LOS-APP-ID', 'write_off_date1_own']]
summary = summary.merge(df5, on='LOS-APP-ID', how='left')
summary = summary.rename(columns = {'write_off_date1_own':'WRITE_OFF_OWN_LATEST_REPORTED_DATE'})

df5_other = data.sort_values('write_off_date1_other').groupby('LOS-APP-ID').head(1)[['LOS-APP-ID', 'write_off_date1_other']]
summary = summary.merge(df5_other, on='LOS-APP-ID', how='left')
summary = summary.rename(columns = {'write_off_date1_other':'WRITE_OFF_OTHERS_LATEST_REPORTED_DATE'})

df_own1 = data.groupby('LOS-APP-ID', as_index=False)['WRITE_OFF_OWN'].max()
summary = summary.merge(df_own1, on='LOS-APP-ID', how='left')

df_other1 = data.groupby('LOS-APP-ID', as_index=False)['WRITE_OFF_OTHERS'].max()
summary = summary.merge(df_other1, on='LOS-APP-ID', how='left')

summary['WRITE_OFF_OWN'] = np.where(summary['WRITE_OFF_OWN'] == 1, 'Y', 'N')
summary['WRITE_OFF_OTHERS'] = np.where(summary['WRITE_OFF_OTHERS'] == 1, 'Y', 'N')

summary['WRITE_OFF_OWN_OWN_LATEST_REPORTED_DATE'] = np.where(summary['WRITE_OFF_OWN'] == 'Y', summary['WRITE_OFF_OWN_LATEST_REPORTED_DATE'], np.nan)
summary['WRITE_OFF_OTHERS_LATEST_REPORTED_DATE'] = np.where(summary['WRITE_OFF_OTHERS'] == 'Y', summary['WRITE_OFF_OTHERS_LATEST_REPORTED_DATE'], np.nan)


In [11]:
summary.columns

Index(['LOS-APP-ID', 'DATE-REPORTED', 'min_year', 'today_date',
       'CREDIT_HISTORY', 'WRITE_OFF_OWN_LATEST_REPORTED_DATE',
       'WRITE_OFF_OTHERS_LATEST_REPORTED_DATE', 'WRITE_OFF_OWN',
       'WRITE_OFF_OTHERS', 'WRITE_OFF_OWN_OWN_LATEST_REPORTED_DATE'],
      dtype='object')

In [12]:
#suite filed 
data['count']=data['DAS - HIST'].str.contains('S16|S23')

data['SUIT_FILED_OWN'] = np.where((data['CREDIT-GRANTOR'] != 'XXXX') & (data['count'] > 0), 1, 0)

data['SUIT_FILED_OTHERS'] = np.where((data['CREDIT-GRANTOR'] == 'XXXX') & (data['count'] > 0), 1, 0)
#-----------
data['s0'] = data['DAS - HIST'].str.find('S16') + 1
data['s1'] = data['DAS - HIST'].str.find('S23') + 1
data['s'] = np.where(data['s0'] > 0, data['s0'], data['s1'])

data[['s','count']]
data[data['s'] > 0]
#print(data['DATE-REPORTED'].head())
data['DATE-REPORTED_s'] = data.apply(lambda x: udf_add_months(str(x['DATE-REPORTED']), x['s']), 
                        axis=1)
# data[data['s'] > 0]
data['suit_filed_date_own'] = np.where((data['s'] != 0) & (data['CREDIT-GRANTOR'] != 'XXXX'), data['DATE-REPORTED_s'], np.nan)

data['suit_filed_date_other'] = np.where((data['s'] != 0) & (data['CREDIT-GRANTOR'] == 'XXXX'), data['DATE-REPORTED_s'], np.nan)

df5 = data.sort_values('suit_filed_date_own').groupby('LOS-APP-ID').head(1)[['LOS-APP-ID', 'suit_filed_date_own']]
summary = summary.merge(df5, on='LOS-APP-ID', how='left')
summary = summary.rename(columns = {'suit_filed_date_own':'SUIT_FILED_OWN_LATEST_REPORTED_DATE'})

df5_other = data.sort_values('suit_filed_date_other').groupby('LOS-APP-ID').head(1)[['LOS-APP-ID', 'suit_filed_date_other']]
summary = summary.merge(df5_other, on='LOS-APP-ID', how='left')
summary = summary.rename(columns = {'suit_filed_date_other':'SUIT_FILED_OTHERS_LATEST_REPORTED_DATE'})

df_own1 = data.groupby('LOS-APP-ID', as_index=False)['SUIT_FILED_OWN'].max()
summary = summary.merge(df_own1, on='LOS-APP-ID', how='left')

df_other1 = data.groupby('LOS-APP-ID', as_index=False)['SUIT_FILED_OTHERS'].max()
summary = summary.merge(df_other1, on='LOS-APP-ID', how='left')

summary['SUIT_FILED_OWN'] = np.where(summary['SUIT_FILED_OWN'] == 1, 'Y', 'N')
summary['SUIT_FILED_OTHERS'] = np.where(summary['SUIT_FILED_OTHERS'] == 1, 'Y', 'N')

summary['SUIT_FILED_OWN_LATEST_REPORTED_DATE'] = np.where(summary['SUIT_FILED_OWN'] == 'Y', summary['SUIT_FILED_OWN_LATEST_REPORTED_DATE'], np.nan)
summary['SUIT_FILED_OTHERS_LATEST_REPORTED_DATE'] = np.where(summary['SUIT_FILED_OTHERS'] == 'Y', summary['SUIT_FILED_OTHERS_LATEST_REPORTED_DATE'], np.nan)


In [13]:
#restructure
data['count']=data['DAS - HIST'].str.contains('S18')

data['RESTRUCTURED_OWN'] = np.where((data['CREDIT-GRANTOR'] != 'XXXX') & (data['count'] > 0), 1, 0)

data['RESTRUCTURED_OTHERS'] = np.where((data['CREDIT-GRANTOR'] == 'XXXX') & (data['count'] > 0), 1, 0)

data['r'] = data['DAS - HIST'].str.find('S18') + 1
print(data['DATE-REPORTED'].head())
data['DATE-REPORTED_r'] = data.apply(lambda x: udf_add_months(str(x['DATE-REPORTED']), x['r']), 
                        axis=1)

data['restructured_date_own'] = np.where((data['r'] != 0) & (data['CREDIT-GRANTOR'] != 'XXXX'), data['DATE-REPORTED_r'], np.nan)

data['restructured_date_other'] = np.where((data['r'] != 0) & (data['CREDIT-GRANTOR'] == 'XXXX'), data['DATE-REPORTED_r'], np.nan)

df5 = data.sort_values('restructured_date_own').groupby('LOS-APP-ID').head(1)[['LOS-APP-ID', 'restructured_date_own']]
summary = summary.merge(df5, on='LOS-APP-ID', how='left')
summary = summary.rename(columns = {'restructured_date_own':'RESTRUCTURED_OWN_LATEST_REPORTED_DATE'})

df5_other = data.sort_values('restructured_date_other').groupby('LOS-APP-ID').head(1)[['LOS-APP-ID', 'restructured_date_other']]
summary = summary.merge(df5_other, on='LOS-APP-ID', how='left')
summary = summary.rename(columns = {'restructured_date_other':'RESTRUCTURED_OTHERS_LATEST_REPORTED_DATE'})

df_rdo = data.groupby('LOS-APP-ID', as_index=False)['RESTRUCTURED_OWN'].max()
summary = summary.merge(df_rdo, on='LOS-APP-ID', how='left')

df_rdoo = data.groupby('LOS-APP-ID', as_index=False)['RESTRUCTURED_OTHERS'].max()
summary = summary.merge(df_rdoo, on='LOS-APP-ID', how='left')

summary['RESTRUCTURED_OWN'] = np.where(summary['RESTRUCTURED_OWN'] == 1, 'Y', 'N')
summary['RESTRUCTURED_OTHERS'] = np.where(summary['RESTRUCTURED_OTHERS'] == 1, 'Y', 'N')

summary['RESTRUCTURED_OWN_LATEST_REPORTED_DATE'] = np.where(summary['RESTRUCTURED_OWN'] == 'Y', summary['RESTRUCTURED_OWN_LATEST_REPORTED_DATE'], np.nan)
summary['RESTRUCTURED_OTHERS_LATEST_REPORTED_DATE'] = np.where(summary['RESTRUCTURED_OTHERS'] == 'Y', summary['RESTRUCTURED_OTHERS_LATEST_REPORTED_DATE'], np.nan)


0   2021-01-05
1   2023-02-28
2   2022-08-31
3   2023-03-31
4   2023-03-31
Name: DATE-REPORTED, dtype: datetime64[ns]


In [14]:
data[data['r']>0]

,CREDT-RPT-ID,LOS-APP-ID,CANDIDATE - ID,CUSTOMER ID/MBR ID,BRANCH,KENDRA,SELF-INDICATOR,MATCH-TYPE,ACC-NUM,CREDIT-GRANTOR,...,s,DATE-REPORTED_s,suit_filed_date_own,suit_filed_date_other,RESTRUCTURED_OWN,RESTRUCTURED_OTHERS,r,DATE-REPORTED_r,restructured_date_own,restructured_date_other
62,NEOG230418CR5117983880502,1199105,161812239,NaN,NaN,NaN,False,PRIMARY,XXXX,XXXX,...,0.0,31/03/2023,NaN,NaN,0,1,88.0,31/08/2025,NaN,31/08/2025
69,NEOG230418CR5117983880502,1199105,870491746,NaN,NaN,NaN,False,PRIMARY,XXXX,XXXX,...,0.0,31/03/2023,NaN,NaN,0,1,88.0,31/08/2025,NaN,31/08/2025
164,NEOG230418CR9217983880502,1202418,6286458672,NaN,NaN,NaN,False,PRIMARY,XXXX,XXXX,...,0.0,31/12/2020,NaN,NaN,0,1,7.0,28/02/2021,NaN,28/02/2021
1137,NEOG230418CR0317983880502,1203696,3555840092,NaN,NaN,NaN,False,PRIMARY,XXXX,XXXX,...,0.0,30/09/2021,NaN,NaN,0,1,1.0,30/09/2021,NaN,30/09/2021
1144,NEOG230418CR0317983880502,1203696,6212932613,NaN,NaN,NaN,False,PRIMARY,XXXX,XXXX,...,0.0,30/06/2021,NaN,NaN,0,1,7.0,30/08/2021,NaN,30/08/2021


In [15]:
summary['RESTRUCTURED_OWN']

0     N
1     N
2     N
3     N
4     N
5     N
6     N
7     N
8     N
9     N
10    N
11    N
12    N
13    N
14    N
15    N
16    N
17    N
18    N
19    N
20    N
21    N
22    N
23    N
24    N
25    N
26    N
27    N
28    N
29    N
30    N
31    N
32    N
33    N
34    N
35    N
36    N
37    N
38    N
39    N
40    N
41    N
42    N
43    N
44    N
45    N
46    N
47    N
48    N
49    N
50    N
51    N
52    N
53    N
Name: RESTRUCTURED_OWN, dtype: object

In [16]:
#*****************************************settled
data['count']=data['DAS - HIST'].str.contains('S19')

data['SETTLED_OWN'] = np.where((data['CREDIT-GRANTOR'] != 'XXXX') & (data['count'] > 0), 1, 0)

data['SETTLED_OTHERS'] = np.where((data['CREDIT-GRANTOR'] == 'XXXX') & (data['count'] > 0), 1, 0)

data['c'] = data['DAS - HIST'].str.find('S19') + 1
print(data['DATE-REPORTED'].head())
data['DATE-REPORTED3'] = data.apply(lambda x: udf_add_months(str(x['DATE-REPORTED']), x['c']), 
                        axis=1)

data['settled_date_own'] = np.where((data['c'] != 0) & (data['CREDIT-GRANTOR'] != 'XXXX'), data['DATE-REPORTED3'], np.nan)

data['settled_date_other'] = np.where((data['c'] != 0) & (data['CREDIT-GRANTOR'] == 'XXXX'), data['DATE-REPORTED3'], np.nan)

df5 = data.sort_values('settled_date_own').groupby('LOS-APP-ID').head(1)[['LOS-APP-ID', 'settled_date_own']]
summary = summary.merge(df5, on='LOS-APP-ID', how='left')
summary = summary.rename(columns = {'settled_date_own':'SETLLED_OWN_LATEST_REPORTED_DATE'})

df5_other = data.sort_values('settled_date_other').groupby('LOS-APP-ID').head(1)[['LOS-APP-ID', 'settled_date_other']]
summary = summary.merge(df5_other, on='LOS-APP-ID', how='left')
summary = summary.rename(columns = {'settled_date_other':'SETLLED_OTHERS_LATEST_REPORTED_DATE'})


df_so = data.groupby('LOS-APP-ID', as_index=False)['SETTLED_OWN'].max()
summary = summary.merge(df_so, on='LOS-APP-ID', how='left')

df_soo = data.groupby('LOS-APP-ID', as_index=False)['SETTLED_OTHERS'].max()
summary = summary.merge(df_soo, on='LOS-APP-ID', how='left')



summary['SETTLED_OWN'] = np.where(summary['SETTLED_OWN'] == 1, 'Y', 'N')
summary['SETTLED_OTHERS'] = np.where(summary['SETTLED_OTHERS'] == 1, 'Y', 'N')

summary['SETLLED_OWN_LATEST_REPORTED_DATE'] = np.where(summary['SETTLED_OWN'] == 'Y', summary['SETLLED_OWN_LATEST_REPORTED_DATE'], np.nan)
summary['SETLLED_OTHERS_LATEST_REPORTED_DATE'] = np.where(summary['SETTLED_OTHERS'] == 'Y', summary['SETLLED_OTHERS_LATEST_REPORTED_DATE'], np.nan)

0   2021-01-05
1   2023-02-28
2   2022-08-31
3   2023-03-31
4   2023-03-31
Name: DATE-REPORTED, dtype: datetime64[ns]


In [17]:
#*********************WILFUL_DEFAULT
data['count']=data['DAS - HIST'].str.contains('S22')

data['WILFUL_DEFAULT_OWN'] = np.where((data['CREDIT-GRANTOR'] != 'XXXX') & (data['count'] > 0), 1, 0)

data['WILFUL_DEFAULT_OTHERS'] = np.where((data['CREDIT-GRANTOR'] != 'XXXX') & (data['count'] > 0), 1, 0)


data['d'] = data['DAS - HIST'].str.find('S22') + 1
data['DATE-REPORTED4'] = data.apply(lambda x: udf_add_months(str(x['DATE-REPORTED']), x['d']), 
                        axis=1)

data['wilfull_default_date_own'] = np.where((data['d'] != 0) & (data['CREDIT-GRANTOR'] != 'XXXX')
                                            , data['DATE-REPORTED4'], np.nan)

data['wilfull_default_date_other'] = np.where((data['d'] != 0) & (data['CREDIT-GRANTOR'] == 'XXXX')
                                            , data['DATE-REPORTED4'], np.nan)


df6_1 = data[['LOS-APP-ID', 'WILFUL_DEFAULT_OWN', 'WILFUL_DEFAULT_OTHERS']]
df6_1 = df6_1.drop_duplicates()

summary = summary.merge(df6_1, on='LOS-APP-ID', how='left')

summary['WILFUL_DEFAULT_OWN'] = np.where(summary['WILFUL_DEFAULT_OWN'] == 1, 'Y', 'N')
summary['WILFUL_DEFAULT_OTHERS'] = np.where(summary['WILFUL_DEFAULT_OTHERS'] == 1, 'Y', 'N')

df6 = data.sort_values('wilfull_default_date_own').groupby('LOS-APP-ID').head(1)[['LOS-APP-ID', 'wilfull_default_date_own']]
summary = summary.merge(df6, on='LOS-APP-ID', how='left')
summary = summary.rename(columns = {'wilfull_default_date_own':'WILFULL_DEFAULT_OWN_LATEST_REPORTED_DATE'})

df6_other = data.sort_values('wilfull_default_date_other').groupby('LOS-APP-ID').head(1)[['LOS-APP-ID', 'wilfull_default_date_other']]
summary = summary.merge(df6_other, on='LOS-APP-ID', how='left')
summary = summary.rename(columns = {'wilfull_default_date_other':'WILFULL_DEFAULT_OTHERS_LATEST_REPORTED_DATE'})

summary['WILFULL_DEFAULT_OWN_LATEST_REPORTED_DATE'] = np.where(summary['WILFUL_DEFAULT_OWN'] == 'Y', summary['WILFULL_DEFAULT_OWN_LATEST_REPORTED_DATE'], np.nan)
summary['WILFULL_DEFAULT_OTHERS_LATEST_REPORTED_DATE'] = np.where(summary['WILFUL_DEFAULT_OTHERS'] == 'Y', summary['WILFULL_DEFAULT_OTHERS_LATEST_REPORTED_DATE'], np.nan)


In [18]:
data['max_default_lender'] = data['DPD - HIST'].apply(udf_dpd)
data['max_default_lender'] = np.where(data['CREDIT-GRANTOR'] != 'XXXX', data['max_default_lender'], 0)

data['max_default_lender'] = data['max_default_lender'].replace('nan', 0)
data['max_default_lender'] = data['max_default_lender'].replace('', 0)
data['max_default_lender'] = data['max_default_lender'].fillna('0')
data['max_default_lender'] = data['max_default_lender'].astype("int")

df7 = data.groupby('LOS-APP-ID', as_index=False)['max_default_lender'].max()
summary = summary.merge(df7, on='LOS-APP-ID', how='left')

data['max_default_other'] = data['DPD - HIST'].apply(udf_dpd)
data['max_default_other'] = np.where(data['CREDIT-GRANTOR'] == 'XXXX', data['max_default_other'], 0)

data['max_default_other'] = data['max_default_other'].replace('nan', 0)
data['max_default_other'] = data['max_default_other'].replace('', 0)
data['max_default_other'] = data['max_default_other'].fillna('0')
data['max_default_other'] = data['max_default_other'].astype("int")

df8 = data.groupby('LOS-APP-ID', as_index=False)['max_default_other'].max()
summary = summary.merge(df8, on='LOS-APP-ID', how='left')
summary = summary.rename(columns = {'max_default_lender':'MAX_DPD_LENDER', 'max_default_other': 'MAX_DPD_OTHERS'})

data['today_date'] = pd.to_datetime('today').strftime("%Y/%m/%d")
data['today_date'] = pd.to_datetime(data['today_date'], format="%Y/%m/%d")
data['diff_months'] = pd.DatetimeIndex(data['today_date']) - pd.DatetimeIndex(data['DATE-REPORTED'])
data['diff_months'] = data['diff_months']/np.timedelta64(1,'M')
data['diff_months']=data['diff_months'].astype(int)


data['last_two_months'] = np.where(data['diff_months'] < 2, data['diff_months']*3, 0)
data['last_three_months'] = np.where(data['diff_months'] < 3, data['diff_months']*3, 0)
data['last_six_months'] = np.where(data['diff_months'] < 6, data['diff_months']*3, 0)
data['last_12_months'] = np.where(data['diff_months'] < 12, data['diff_months']*3, 0)
data['last_24_months'] = np.where(data['diff_months'] < 24, data['diff_months']*3, 0)

data['dpd_2months'] = data.apply(lambda x: udf_dpd_months(x['DPD - HIST'], x['last_two_months']), axis=1)
data['dpd_3months'] = data.apply(lambda x: udf_dpd_months(x['DPD - HIST'], x['last_three_months']), axis=1)
data['dpd_6months'] = data.apply(lambda x: udf_dpd_months(x['DPD - HIST'], x['last_six_months']), axis=1)
data['dpd_12months'] = data.apply(lambda x: udf_dpd_months(x['DPD - HIST'], x['last_12_months']), axis=1)
data['dpd_24months'] = data.apply(lambda x: udf_dpd_months(x['DPD - HIST'], x['last_24_months']), axis=1)


print(data['dpd_2months'].apply(udf_dpd).head())
data['max_default_lender_2months'] = data['dpd_2months'].apply(udf_dpd)
data['max_default_lender_2months'] = np.where(data['CREDIT-GRANTOR'] != 'XXXX', data['max_default_lender_2months'], 0)

data['max_default_lender_2months'] = data['max_default_lender_2months'].replace('nan', 0)
data['max_default_lender_2months'] = data['max_default_lender_2months'].replace('', 0)
data['max_default_lender_2months'] = data['max_default_lender_2months'].replace('.', '')
data['max_default_lender_2months'] = data['max_default_lender_2months'].fillna('0')
data['max_default_lender_2months'] = data['max_default_lender_2months'].astype("int")

df8 = data.groupby('LOS-APP-ID', as_index=False)['max_default_lender_2months'].max()
summary = summary.merge(df8, on='LOS-APP-ID', how='left')



data['max_default_other_2months'] = data['dpd_2months'].apply(udf_dpd)
data['max_default_other_2months'] = np.where(data['CREDIT-GRANTOR'] == 'XXXX', data['max_default_other_2months'], 0)

data['max_default_other_2months'] = data['max_default_other_2months'].replace('nan', 0)
data['max_default_other_2months'] = data['max_default_other_2months'].replace('', 0)
data['max_default_lender_2months'] = data['max_default_lender_2months'].replace('.', '')
data['max_default_other_2months'] = data['max_default_other_2months'].fillna('0')
data['max_default_other_2months'] = data['max_default_other_2months'].astype("int")

df9 = data.groupby('LOS-APP-ID', as_index=False)['max_default_other_2months'].max()
summary = summary.merge(df9, on='LOS-APP-ID', how='left')

data['max_default_lender_3months'] = data['dpd_3months'].apply(udf_dpd)
data['max_default_lender_3months'] = np.where(data['CREDIT-GRANTOR'] != 'XXXX', data['max_default_lender_3months'], 0)

data['max_default_lender_3months'] = data['max_default_lender_3months'].replace('nan', 0)
data['max_default_lender_3months'] = data['max_default_lender_3months'].replace('', 0)
data['max_default_lender_2months'] = data['max_default_lender_2months'].replace('.', '')
data['max_default_lender_3months'] = data['max_default_lender_3months'].fillna('0')
data['max_default_lender_3months'] = data['max_default_lender_3months'].astype("int")

df10 = data.groupby('LOS-APP-ID', as_index=False)['max_default_lender_3months'].max()
summary = summary.merge(df10, on='LOS-APP-ID', how='left')

data['max_default_other_3months'] = data['dpd_3months'].apply(udf_dpd)
data['max_default_other_3months'] = np.where(data['CREDIT-GRANTOR'] == 'XXXX', data['max_default_other_3months'],0)

data['max_default_other_3months'] = data['max_default_other_3months'].replace('nan', 0)
data['max_default_other_3months'] = data['max_default_other_3months'].replace('', 0)
data['max_default_lender_2months'] = data['max_default_lender_2months'].replace('.', '')
data['max_default_other_3months'] = data['max_default_other_3months'].fillna('0')
data['max_default_other_3months'] = data['max_default_other_3months'].astype("int")

df11 = data.groupby('LOS-APP-ID', as_index=False)['max_default_other_3months'].max()
summary = summary.merge(df11, on='LOS-APP-ID', how='left')


data['max_default_lender_6months'] = data['dpd_6months'].apply(udf_dpd)
data['max_default_lender_6months'] = np.where(data['CREDIT-GRANTOR'] != 'XXXX', data['max_default_lender_6months'], 0)

data['max_default_lender_6months'] = data['max_default_lender_6months'].replace('nan', 0)
data['max_default_lender_6months'] = data['max_default_lender_6months'].replace('', 0)
data['max_default_lender_2months'] = data['max_default_lender_2months'].replace('.', '')
data['max_default_lender_6months'] = data['max_default_lender_6months'].fillna('0')
data['max_default_lender_6months'] = data['max_default_lender_6months'].astype("int")

df12 = data.groupby('LOS-APP-ID', as_index=False)['max_default_lender_6months'].max()
summary = summary.merge(df12, on='LOS-APP-ID', how='left')

data['max_default_other_6months'] = data['dpd_6months'].apply(udf_dpd)
data['max_default_other_6months'] = np.where(data['CREDIT-GRANTOR'] == 'XXXX', data['max_default_other_6months'],0)

data['max_default_other_6months'] = data['max_default_other_6months'].replace('nan', 0)
data['max_default_other_6months'] = data['max_default_other_6months'].replace('', 0)
data['max_default_lender_2months'] = data['max_default_lender_2months'].replace('.', '')
data['max_default_other_6months'] = data['max_default_other_6months'].fillna('0')
data['max_default_other_6months'] = data['max_default_other_6months'].astype("int")

df13 = data.groupby('LOS-APP-ID', as_index=False)['max_default_other_6months'].max()
summary = summary.merge(df13, on='LOS-APP-ID', how='left')



data['max_default_lender_12months'] = data['dpd_12months'].apply(udf_dpd)
data['max_default_lender_12months'] = np.where(data['CREDIT-GRANTOR'] != 'XXXX', data['max_default_lender_12months'], 0)

data['max_default_lender_12months'] = data['max_default_lender_12months'].replace('nan', 0)
data['max_default_lender_12months'] = data['max_default_lender_12months'].replace('', 0)
data['max_default_lender_2months'] = data['max_default_lender_2months'].replace('.', '')
data['max_default_lender_12months'] = data['max_default_lender_12months'].fillna('0')
data['max_default_lender_12months'] = data['max_default_lender_12months'].astype("int")

df14 = data.groupby('LOS-APP-ID', as_index=False)['max_default_lender_12months'].max()
summary = summary.merge(df14, on='LOS-APP-ID', how='left')

data['max_default_other_12months'] = data['dpd_12months'].apply(udf_dpd)
data['max_default_other_12months'] = np.where(data['CREDIT-GRANTOR'] == 'XXXX', data['max_default_other_12months'],0)

data['max_default_other_12months'] = data['max_default_other_12months'].replace('nan', 0)
data['max_default_other_12months'] = data['max_default_other_12months'].replace('', 0)
data['max_default_lender_2months'] = data['max_default_lender_2months'].replace('.', '')
data['max_default_other_12months'] = data['max_default_other_12months'].fillna('0')
data['max_default_other_12months'] = data['max_default_other_12months'].astype("int")

df15 = data.groupby('LOS-APP-ID', as_index=False)['max_default_other_12months'].max()
summary = summary.merge(df15, on='LOS-APP-ID', how='left')


data['max_default_lender_24months'] = data['dpd_24months'].apply(udf_dpd)
data['max_default_lender_24months'] = np.where(data['CREDIT-GRANTOR'] != 'XXXX', data['max_default_lender_24months'], 0)

data['max_default_lender_24months'] = data['max_default_lender_24months'].replace('nan', 0)
data['max_default_lender_24months'] = data['max_default_lender_24months'].replace('', 0)
data['max_default_lender_2months'] = data['max_default_lender_2months'].replace('.', '')
data['max_default_lender_24months'] = data['max_default_lender_24months'].fillna('0')
data['max_default_lender_24months'] = data['max_default_lender_24months'].astype("int")

df16 = data.groupby('LOS-APP-ID', as_index=False)['max_default_lender_24months'].max()
summary = summary.merge(df16, on='LOS-APP-ID', how='left')

data['max_default_other_24months'] = data['dpd_24months'].apply(udf_dpd)
data['max_default_other_24months'] = np.where(data['CREDIT-GRANTOR'] == 'XXXX', data['max_default_other_24months'], 0)

data['max_default_other_24months'] = data['max_default_other_24months'].replace('nan', 0)
data['max_default_other_24months'] = data['max_default_other_24months'].replace('', 0)
data['max_default_lender_2months'] = data['max_default_lender_2months'].replace('.', '')
data['max_default_other_24months'] = data['max_default_other_24months'].fillna('0')
data['max_default_other_24months'] = data['max_default_other_24months'].astype("int")

df17 = data.groupby('LOS-APP-ID', as_index=False)['max_default_other_24months'].max()
summary = summary.merge(df17, on='LOS-APP-ID', how='left')

summary = summary.rename(columns = {'max_default_lender_2months':'MAX_DPD_LENDER_2months', 
                                    'max_default_other_2months': 'MAX_DPD_OTHERS_2months',
                                   'max_default_lender_3months':'MAX_DPD_LENDER_3months', 
                                    'max_default_other_3months': 'MAX_DPD_OTHERS_3months',
                                   'max_default_lender_6months':'MAX_DPD_LENDER_6months', 
                                    'max_default_other_6months': 'MAX_DPD_OTHERS_6months',
                                   'max_default_lender_12months':'MAX_DPD_LENDER_12months', 
                                    'max_default_other_12months': 'MAX_DPD_OTHERS_12months',
                                   'max_default_lender_24months':'MAX_DPD_LENDER_24months', 
                                    'max_default_other_24months': 'MAX_DPD_OTHERS_24months'})

data['max_dpd_all'] = data['DPD - HIST'].apply(udf_dpd)

data['max_dpd_all'] = data['max_dpd_all'].replace('nan', 0)
data['max_dpd_all'] = data['max_dpd_all'].replace('', 0)
data['max_dpd_all'] = data['max_dpd_all'].fillna('0')
data['max_dpd_all'] = data['max_dpd_all'].astype("int")

df18 = data.groupby('LOS-APP-ID', as_index=False)['max_dpd_all'].max()
summary = summary.merge(df18, on='LOS-APP-ID', how='left')


status_to_temp_dict = { 'Sold/Purchased':1, 'Current Account': 2, 'Active': 2, 'Delinquent': 3, 'Restructured': 4,
                      'Settled': 5, 'Written Off': 6, 'Suit Filed': 7, 'WILFUL DEFAULT': 7, 'Closed Account': 0,
                      'Closed': 0, 'SUIT FILED (WILFUL DEFAULT)' : 7, 'Cancelled': 0, 'Loan Declined': 8,'Loan Approved - Not yet disbursed':2,
                      'Loan Submitted':2, 'Restructured  and  Closed': 4,'Post Write Off Closed':6,'Post Write Off Settled':6}

temp_to_status_dict = { 1:'Sold/Purchased', 2: 'Current Account',3: 'Delinquent', 4:'Restructured',
                      5:'Settled', 6:'Written Off', 7:'Suit Filed', 0:'Closed Account', 8 : 'Loan Declined'}


data['dpd_status_numbers_own'] = data['ACCOUNT-STATUS']#.apply(udf_status_to_temp)
data['dpd_status_numbers_others'] = data['ACCOUNT-STATUS']
data = data.replace({"dpd_status_numbers_own": status_to_temp_dict})
data = data.replace({"dpd_status_numbers_others": status_to_temp_dict})

0      0
1    000
2      0
3      0
4      0
Name: dpd_2months, dtype: object


In [19]:
data['dpd_status_numbers_own'] = np.where(data['CREDIT-GRANTOR'] != 'XXXX', data['dpd_status_numbers_own'], -99)
#data.to_csv('Arohan_data.csv',index = False)
print(data['dpd_status_numbers_own'].max())
df19 = data.groupby('LOS-APP-ID', as_index=False)['dpd_status_numbers_own'].max()
summary = summary.merge(df19, on='LOS-APP-ID', how='left')
summary = summary.rename(columns = {'dpd_status_numbers_own':'DPD_STATUS_OWN'})

data['dpd_status_numbers_others'] = np.where(data['CREDIT-GRANTOR'] == 'XXXX', data['dpd_status_numbers_others'], -99)
df20 = data.groupby('LOS-APP-ID', as_index=False)['dpd_status_numbers_others'].max()
summary = summary.merge(df20, on='LOS-APP-ID', how='left')
summary = summary.rename(columns = {'dpd_status_numbers_others':'DPD_STATUS_OTHERS'})

summary = summary.replace({"DPD_STATUS_OWN": temp_to_status_dict})
summary = summary.replace({"DPD_STATUS_OTHERS": temp_to_status_dict})

summary['DPD_STATUS_OWN'] = np.where(summary['DPD_STATUS_OWN'] == -99, np.nan, summary['DPD_STATUS_OWN'])
summary['DPD_STATUS_OTHERS'] = np.where(summary['DPD_STATUS_OTHERS'] == -99, np.nan, summary['DPD_STATUS_OTHERS'])

data['COUNT_30DPD_OWN'] = data['DPD - HIST'].apply(udf_dpd_30)
data['COUNT_30DPD_OWN'] = np.where(data['CREDIT-GRANTOR'] != 'XXXX', data['COUNT_30DPD_OWN'], 0)

df22 = data.groupby('LOS-APP-ID', as_index=False)['COUNT_30DPD_OWN'].sum()
summary = summary.merge(df22, on='LOS-APP-ID', how='left')

data['bad'] = data['DPD - HIST'].apply(udf_dpd_bad)
data['bad'] = np.where(data['CREDIT-GRANTOR'] != 'XXXX', data['bad'], 0)

data['CURRENT-BAL'] = data['CURRENT-BAL'].astype("string")
data['CURRENT-BAL'] = data['CURRENT-BAL'].str.replace(',','')

data['CURRENT-BAL'] = data['CURRENT-BAL'].replace('nan', 0)
data['CURRENT-BAL'] = data['CURRENT-BAL'].replace('', 0)
data['CURRENT-BAL'] = data['CURRENT-BAL'].fillna('0')
data['CURRENT-BAL'] = data['CURRENT-BAL'].astype("int")

data['OVERDUE-AMT'] = data['OVERDUE-AMT'].astype("string")
data['OVERDUE-AMT'] = data['OVERDUE-AMT'].str.replace(',','')

data['OVERDUE-AMT'] = data['OVERDUE-AMT'].replace('nan', 0)
data['OVERDUE-AMT'] = data['OVERDUE-AMT'].replace('', 0)
data['OVERDUE-AMT'] = data['OVERDUE-AMT'].fillna('0')
data['OVERDUE-AMT'] = data['OVERDUE-AMT'].astype("int")



2


In [20]:
df21 = (data.groupby(['LOS-APP-ID', 'ACCOUNT-STATUS'])['CURRENT-BAL']
         .agg([('Count','size'), ('current_bal','sum')])
         .reset_index())
df21

,LOS-APP-ID,ACCOUNT-STATUS,Count,current_bal
0,1166440,Active,15,22271352
1,1166440,Closed,5,-685
2,1172480,Active,19,16955733
3,1172480,Closed,3,588
4,1172801,Active,19,49092212
...,...,...,...,...
134,1209388,Active,24,5947760
135,1209388,Closed,40,20103
136,1209388,Delinquent,1,52229
137,1209418,Active,34,4488865


In [21]:
options = ['Closed', 'Settled', 'Cancelled', 'Written Of']
df22 = df21.loc[~df21['ACCOUNT-STATUS'].isin(options)] 
df22[df22['LOS-APP-ID']=='ZL1315187776']

,LOS-APP-ID,ACCOUNT-STATUS,Count,current_bal


In [22]:

df23 = df22.groupby('LOS-APP-ID', as_index=False)['current_bal'].sum()
summary = summary.merge(df23, on='LOS-APP-ID', how='left')
summary = summary.rename(columns = {'current_bal':'POS'})

df24 = df22.groupby('LOS-APP-ID', as_index=False)['Count'].sum()
summary = summary.merge(df24, on='LOS-APP-ID', how='left')
summary = summary.rename(columns = {'Count':'NO_OF_ACTIVE_LOANS'})

data['RESTRUCTURE_OWN_FLAG'] = np.where((data['CREDIT-GRANTOR'] != 'XXXX') 
                                        & (data['ACCOUNT-STATUS'] == 'Restructured'), 1, 0)
data['NO_OF_IRREGULAR_LOANS'] = np.where(data['bad']>0,1,0)

df25 = data.groupby('LOS-APP-ID', as_index=False)['RESTRUCTURE_OWN_FLAG'].sum()
summary = summary.merge(df25, on='LOS-APP-ID', how='left')

df26 = data.groupby('LOS-APP-ID', as_index=False)['NO_OF_IRREGULAR_LOANS'].sum()
summary = summary.merge(df26, on='LOS-APP-ID', how='left')

data['NPA'] = np.where(data['ACCOUNT-STATUS'] != 'Closed',np.where(data['max_dpd_all'] >= 90, data['OVERDUE-AMT'], np.nan), np.nan)

df26 = data.groupby('LOS-APP-ID', as_index=False)['NPA'].sum()
summary = summary.merge(df26, on='LOS-APP-ID', how='left')


In [23]:
summary.columns

Index(['LOS-APP-ID', 'DATE-REPORTED', 'min_year', 'today_date',
       'CREDIT_HISTORY', 'WRITE_OFF_OWN_LATEST_REPORTED_DATE',
       'WRITE_OFF_OTHERS_LATEST_REPORTED_DATE', 'WRITE_OFF_OWN',
       'WRITE_OFF_OTHERS', 'WRITE_OFF_OWN_OWN_LATEST_REPORTED_DATE',
       'SUIT_FILED_OWN_LATEST_REPORTED_DATE',
       'SUIT_FILED_OTHERS_LATEST_REPORTED_DATE', 'SUIT_FILED_OWN',
       'SUIT_FILED_OTHERS', 'RESTRUCTURED_OWN_LATEST_REPORTED_DATE',
       'RESTRUCTURED_OTHERS_LATEST_REPORTED_DATE', 'RESTRUCTURED_OWN',
       'RESTRUCTURED_OTHERS', 'SETLLED_OWN_LATEST_REPORTED_DATE',
       'SETLLED_OTHERS_LATEST_REPORTED_DATE', 'SETTLED_OWN', 'SETTLED_OTHERS',
       'WILFUL_DEFAULT_OWN', 'WILFUL_DEFAULT_OTHERS',
       'WILFULL_DEFAULT_OWN_LATEST_REPORTED_DATE',
       'WILFULL_DEFAULT_OTHERS_LATEST_REPORTED_DATE', 'MAX_DPD_LENDER',
       'MAX_DPD_OTHERS', 'MAX_DPD_LENDER_2months', 'MAX_DPD_OTHERS_2months',
       'MAX_DPD_LENDER_3months', 'MAX_DPD_OTHERS_3months',
       'MAX_DPD_LENDER_6mont

In [24]:

#adding history count : # of loans the borrower borrowed irrespective of Active or Inactive/repaid
hist_df = data.groupby('LOS-APP-ID', as_index=False).size()
hist_df = hist_df.rename(columns = {'size':'Record Count'})
summary = summary.merge(hist_df, on = 'LOS-APP-ID', how = 'left')#adding history count

#Adding Cibil score into Summary data
inp_summary = pd.read_csv("./NeoGrowth_CRIF format_17-04-2023_Output/Summary.csv",sep='|',index_col=False)
inp_summary = inp_summary.rename(columns=lambda x: x.strip())
inp_summary['LOS-APP-ID'] = inp_summary['LOS-APP-ID'].astype('str')
inp_summary.to_csv('Summary.csv',index = False)
summary = summary.merge(inp_summary, how ='left', on='LOS-APP-ID')
summary = summary.rename(columns={'PERFORM_CNS SCORE':'CIBIL_SCORE'})

# NPA Status column add 
summary['NPA-STATUS']  = np.where(summary['NPA'] > 0,'Yes', 'No') 

summary = summary[['LOS-APP-ID', 'CREDIT_HISTORY', 
       'WRITE_OFF_OWN', 'WRITE_OFF_OTHERS', 'WRITE_OFF_OWN_LATEST_REPORTED_DATE', 'WRITE_OFF_OTHERS_LATEST_REPORTED_DATE',
       'SUIT_FILED_OWN', 'SUIT_FILED_OTHERS', 'SUIT_FILED_OWN_LATEST_REPORTED_DATE','SUIT_FILED_OTHERS_LATEST_REPORTED_DATE',
       'SETTLED_OWN', 'SETTLED_OTHERS', 'SETLLED_OWN_LATEST_REPORTED_DATE', 'SETLLED_OTHERS_LATEST_REPORTED_DATE', 
       'WILFUL_DEFAULT_OWN', 'WILFUL_DEFAULT_OTHERS', 'WILFULL_DEFAULT_OWN_LATEST_REPORTED_DATE', 'WILFULL_DEFAULT_OTHERS_LATEST_REPORTED_DATE', 'MAX_DPD_LENDER',
       'RESTRUCTURED_OWN','RESTRUCTURED_OTHERS','RESTRUCTURED_OWN_LATEST_REPORTED_DATE','RESTRUCTURED_OTHERS_LATEST_REPORTED_DATE',        'MAX_DPD_OTHERS', 'MAX_DPD_LENDER_2months', 'MAX_DPD_OTHERS_2months',
       'MAX_DPD_LENDER_3months', 'MAX_DPD_OTHERS_3months',
       'MAX_DPD_LENDER_6months', 'MAX_DPD_OTHERS_6months',
       'MAX_DPD_LENDER_12months', 'MAX_DPD_OTHERS_12months',
       'MAX_DPD_LENDER_24months', 'MAX_DPD_OTHERS_24months', 'max_dpd_all',
       'DPD_STATUS_OWN', 'DPD_STATUS_OTHERS', 'COUNT_30DPD_OWN', 'POS',
       'NO_OF_ACTIVE_LOANS', 'NO_OF_IRREGULAR_LOANS', 'RESTRUCTURE_OWN_FLAG', 'NPA','NPA-STATUS','Record Count','CIBIL_SCORE']]

summary.to_excel("./NeoGrowth_CRIF_Final.xlsx")


In [25]:
summary.shape[0]

54

In [26]:
#summary = summary.drop(['CIBIL_SCORE'], axis=1)
summary.columns

Index(['LOS-APP-ID', 'CREDIT_HISTORY', 'WRITE_OFF_OWN', 'WRITE_OFF_OTHERS',
       'WRITE_OFF_OWN_LATEST_REPORTED_DATE',
       'WRITE_OFF_OTHERS_LATEST_REPORTED_DATE', 'SUIT_FILED_OWN',
       'SUIT_FILED_OTHERS', 'SUIT_FILED_OWN_LATEST_REPORTED_DATE',
       'SUIT_FILED_OTHERS_LATEST_REPORTED_DATE', 'SETTLED_OWN',
       'SETTLED_OTHERS', 'SETLLED_OWN_LATEST_REPORTED_DATE',
       'SETLLED_OTHERS_LATEST_REPORTED_DATE', 'WILFUL_DEFAULT_OWN',
       'WILFUL_DEFAULT_OTHERS', 'WILFULL_DEFAULT_OWN_LATEST_REPORTED_DATE',
       'WILFULL_DEFAULT_OTHERS_LATEST_REPORTED_DATE', 'MAX_DPD_LENDER',
       'RESTRUCTURED_OWN', 'RESTRUCTURED_OTHERS',
       'RESTRUCTURED_OWN_LATEST_REPORTED_DATE',
       'RESTRUCTURED_OTHERS_LATEST_REPORTED_DATE', 'MAX_DPD_OTHERS',
       'MAX_DPD_LENDER_2months', 'MAX_DPD_OTHERS_2months',
       'MAX_DPD_LENDER_3months', 'MAX_DPD_OTHERS_3months',
       'MAX_DPD_LENDER_6months', 'MAX_DPD_OTHERS_6months',
       'MAX_DPD_LENDER_12months', 'MAX_DPD_OTHERS_12months',


In [27]:
test1 = summary[['LOS-APP-ID']]
y = test1.shape[0]
if x == y :
    print("matching!")

matching!


In [28]:
x,y

(54, 54)